In [ ]:
from fuzzywuzzy import process, fuzz
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


os.chdir('/Users/danielmederer/Data Storage Folder')
os.getcwd()


In [ ]:
######### Guide to Script

# First a mapping between TRY names and eyadata(Ecosis) names has to be created
# This mapping ensures that the two datasets use the same naming scheme
# Mapping has to be done with eyadata and TRY hierarchy data
# Once the mapping is done once, the processing steps for TRY and Ecosis can be done separately 
# and can be repeated for adjustments


In [ ]:
## Eyadata loading 

data = pd.read_csv('/Users/danielmederer/Data Storage Folder/test_new.csv', sep = ",", encoding = 'latin-1')

# remove unused columns and rows with NA species
data.drop(['Unnamed: 0', 'dataset', 'Site', 'Year', 'numSamples', 'LandCover', 'Tool'],axis=1,inplace=True)
data.dropna(subset=["Species"],inplace=True)
data.drop(data.iloc[:, 30:], axis = 1, inplace=True)
data.sample(15)

In [ ]:
# Load hierarchy data for matching

hierarchy_data = pd.read_csv('/Users/danielmederer/Data Storage Folder/hierarchy.info.csv', sep = ",", encoding='latin_1')
hierarchy_data.drop(['Unnamed: 0', 'ObservationID'],axis=1,inplace=True)

hierarchy_data

In [ ]:
# Ecosis Species + new names
list_of_strings = data.dropna(subset=["Species"])["Species"].unique()
#dataset['Species'].unique()
# inter.dropna(subset=["Species"])["Species"].unique() 
mapp=[]

####Try names
names=hierarchy_data.dropna(subset=["Species"])["Species"].unique()

for i in range(hierarchy_data['Species'].nunique()):
    print(i)
    mapp.append((process.extract(names[i], list_of_strings, limit=1)[0])+(names[i],))    
mapp

In [ ]:
tr_name = list(zip(*mapp))

traits_df = pd.DataFrame({"EcosisName":tr_name[0],'ratio':tr_name[1],'TryName':tr_name[2]})

traits_df

In [ ]:
## Clean and select the mapping ##

#tuples with ratio >89
mapping=traits_df[traits_df['ratio']>89]
mapping.sort_values(by=['ratio'],inplace=True) #####Sort the tuples according to the ration
mapping

In [ ]:
data['Species'].nunique()

In [ ]:
#####Drop duplicate duples and keep the one with the highest ratio values:last has most of the time the highest ratio
mapping.drop_duplicates(subset=['EcosisName'], keep='last',inplace=True) 

In [ ]:
#####check for duplicates
ids = mapping["EcosisName"]
mapping[ids.isin(ids[ids.duplicated(keep=False)])]

In [ ]:
###drop null values in EcosisName
mapping.dropna(subset=["EcosisName"],inplace=True)

In [ ]:
mapping

In [ ]:
mapping.to_csv('SpeciesMapping_test_NotNullMore89.csv', index = False)

In [ ]:
mapping = pd.read_csv('/Users/danielmederer/Data Storage Folder/SpeciesMapping_test_NotNullMore89.csv',
                      sep = ",")
#mapping.drop(['Unnamed: 0'],axis=1,inplace=True)


In [ ]:
mapping

In [ ]:
### Hierarchy mapping join

#Add the eya mapping name to hierarchy database
F=mapping.merge(hierarchy_data, left_on='TryName',
    right_on='Species', how='left')

####Number of unique Species names for the mapping
print(F['TryName'].nunique()) 
print(F['EcosisName'].nunique()) 

F

In [ ]:
### Ecosis mapping join 

#Add the mapping to eyadata
D = data.merge(mapping, left_on='Species',
    right_on='EcosisName', how='left')

####Number of unique Species names for the mapping
print(D['TryName'].nunique()) 
print(D['EcosisName'].nunique())

D

In [ ]:
D.columns

In [ ]:
## Processing of Eya's data ##

# Remove NAs, both columns and rows (if all are NA)
# columns
eyadata_processed = D[pd.notnull(D['TryName'])]
# rows
eyadata_processed = eyadata_processed.dropna(axis = 0, how = 'all', subset = ['Al content (mg/cmÂ²)', 'Aluminum concentration (mg/g)',
       'Anthocyanin content (Î¼g/cmÂ²)', 'Anthocyanin concentration (mg/g)',
       'Boron content (mg/cmÂ²)', 'Boron concentration (mg/g)',
       'C content (mg/cmÂ²)', 'C concentration (mg/g)', 'Ca content (mg/cmÂ²)',
       'Ca concentration (mg/g)', 'Carotenoid content (Î¼g/cmÂ²)',
       'Carotenoid concentration (mg/g)', 'Cellulose (mg/cmÂ²)',
       'Cellulose (mg/g)', 'Chl content (Î¼g/cmÂ²)',
       'Chlorophyll concentration (mg/g)', 'Copper content (mg/cmÂ²)',
       'Copper concentration (mg/g)', 'EWT (mg/cmÂ²)', 'Fiber (mg/cmÂ²)',
       'Fiber (mg/g)', 'Flavonoids concentration (mg/g)',
       'Flavonoids content (mg/cmÂ²)', 'Iron content (mg/cmÂ²)',
       'Iron concentration (mg/g)', 'LAI (mÂ²/mÂ²)', 'LMA (g/mÂ²)',
       'LDMC (g/g)', 'LWC (%)'])
####Number of not null Species names for the mapping
print(eyadata_processed['TryName'].notnull().sum())

# Remove Species column
eyadata_processed.drop(['Species'],axis=1,inplace=True)

# Make TryName the first column
eyadata_processed.insert(0, 'Species', eyadata_processed['TryName'])
# Remove rest of unnecessary columns
eyadata_processed.drop(['EcosisName', 'ratio', 'TryName'],axis=1,inplace=True)


eyadata_processed

In [ ]:
eyadata_processed['Species'].nunique()

eyadata_processed.to_csv('eyadata_processed_finished1.csv', index = False)

In [ ]:
################ 
################
# Only relevant for Eya's data 
eyadata_processed = pd.read_csv('/Users/danielmederer/Data Storage Folder/eyadata_processed_finished1.csv')

hierarchy_data

In [ ]:
eyadata_processed

In [ ]:
### Hierarchy join with eyadata 

# Remove duplicates of Hierarchy (so they are not used in merge)
hierarchy_data.drop_duplicates(inplace=True)

#Add the eyadata to hierarchy_data
hierarchy_processed = eyadata_processed.merge(hierarchy_data, left_on='Species',
    right_on='Species', how='left')

####Number of unique Species names for the mapping
print(hierarchy_processed['Species'].nunique()) 

hierarchy_processed

In [ ]:
hierarchy_processed = hierarchy_processed.loc[:, ['Species', 'Genus', 'Family']]

In [ ]:
hierarchy_processed.sample(10)

In [ ]:
hierarchy_processed.to_csv('hierarchy_processed.csv', index = True)

In [ ]:
#########
#########
#########

In [ ]:

######## TRYdata ########

data = pd.read_csv('/Users/danielmederer/Data Storage Folder/TRY_processed_unfinished.csv', sep = ",")

data.sample(15)

# Load hierarchy data for matching

hierarchy_data = pd.read_csv('/Users/danielmederer/Data Storage Folder/hierarchy.info.csv', sep = ",", encoding='latin_1')
hierarchy_data.drop(['Unnamed: 0', 'ObservationID'],axis=1,inplace=True)

## Specific for TRY data ##

data.columns


In [ ]:
# optional: subset TRY for shorter processing times

data = data.iloc[:, [0, 1, 2, 4, 5, 8, 9, 10, 12, 14, 15, 16, 17, 18, 19, 20, 21, 25, 26]]
#data = data.iloc[:, [0, 1, 2, 8, 10, 12, 14]]

data

In [ ]:
# Remove rows with all NA (due to subset) => make subset from first to last variable of interest
c = data.loc[:0, 'Dispersal unit length':'Wood vessel element length; stem conduit (vessel and tracheids) element length'].columns  # retrieve only the 0th row for efficiency
data = data.dropna(axis = 0, subset = c, how = 'all') 
data

In [ ]:

# Merge TRYdata with Hierarchy data
D = data.merge(hierarchy_data, left_on='Species',
    right_on='Species', how='left')

####Number of unique Species names for the mapping
print(D['Species'].nunique()) 
D

In [ ]:
print(D['ObservationID'].nunique()) 

In [ ]:
print(D['ObsDataID'].nunique()) 

In [ ]:
# Drop all duplicate observations that were produced by the merging based on ObsDataID (!)

D.drop_duplicates(subset = 'ObsDataID', keep = "first", inplace = True)

D

In [ ]:
# Remove NAs, both columns and rows (if all are NA)

# columns
TRYdata_processed = D[pd.notnull(D['Species'])]
# rows
TRYdata_processed = TRYdata_processed.dropna(axis = 0, how = 'all') 

# Remove rows that contain zeros
TRYdata_processed = TRYdata_processed[(TRYdata_processed != 0).all(1)]

## Number of non-null Species names for the mapping
print(TRYdata_processed['Species'].notnull().sum())

# Remove Species column
#TRYdata_processed.drop(['Species'],axis=1,inplace=True)

# Make TryName the first column
#TRYdata_processed.insert(0, 'Species', TRYdata_processed['TryName'])
# Remove rest of unnecessary columns
#TRYdata_processed.drop(['EcosisName', 'ratio', 'TryName'],axis=1,inplace=True)


TRYdata_processed

In [ ]:
# Rename columns

TRYdata_processed.columns = ["Species", "ObservationID", "ObsDataID", "DispersalUL_mm", "LA_mm2", 
                             "SLA_mm2mg", "carbonc_mgg", "chlc_mygcm2", 
                      "LDMC_gg", "nitrogenc_gm2", "nitrogenc_mgg", "D15N_permill", 
                      "N_P_ratio_gg", "phosphorc_mgg", "LWC_LDM_notsat_gg", "plant_height_m", 
                      "seed_mass_mg", "SSD_gcm3", "ConduitEL_mym", "Genus", "Family"]

#TRYdata_processed.columns = ["Species", "ObservationID", "ObsDataID", "SLA_mm2mg", "chlc_mygcm2", 
                             #"LDMC_gg", "nitrogenc_gm2", "Genus", "Family"]

In [ ]:
TRYdata_processed.sample(10)

In [ ]:
# group dataframe by observation to have all entries of one observation in one row

grouped_df = TRYdata_processed.groupby('ObservationID').agg({'Species': 'first',
                                                'DispersalUL_mm': 'first',
                                                'LA_mm2': 'first',
                                                'SLA_mm2mg': 'first',
                                                'carbonc_mgg': 'first',
                                                'chlc_mygcm2': 'first',
                                                'LDMC_gg': 'first',
                                                'nitrogenc_gm2': 'first',
                                                'nitrogenc_mgg': 'first',             
                                                'D15N_permill': 'first',
                                                'N_P_ratio_gg': 'first',
                                                'phosphorc_mgg': 'first',
                                                'LWC_LDM_notsat_gg': 'first',
                                                'plant_height_m': 'first',
                                                'seed_mass_mg': 'first',
                                                'SSD_gcm3': 'first',
                                                'ConduitEL_mym': 'first',
                                                'Genus': 'first',
                                                'Family': 'first',
                                                            })
'''grouped_df = TRYdata_processed.groupby('ObservationID').agg({'Species': 'first',
                                                'SLA_mm2mg': 'first',
                                                'chlc_mygcm2': 'first',
                                                'LDMC_gg': 'first',
                                                'nitrogenc_gm2': 'first',
                                                'Genus': 'first',
                                                'Family': 'first',
                                                            })'''



In [ ]:
grouped_df

In [ ]:
# make and save dataframe for trait values

TRY_values_processed = grouped_df.iloc[:, 1:17] # adapt for subsets!
TRY_values_processed.to_csv('TRY_values_processed_finished.csv', index = False)

In [ ]:
TRY_values_processed

In [ ]:
# make and save dataframe for hierarchy
TRY_hierarchy_processed = grouped_df[["Species", "Genus", "Family"]]
TRY_hierarchy_processed.to_csv('TRY_hierarchy_processed_finished.csv', index = True) # True to create index column

In [ ]:
TRY_hierarchy_processed

In [ ]:
## General saving scripts ##

grouped_df['Species'].nunique()

grouped_df.to_csv('TRYdata_processed_finished.csv', index = False)